# Extração de dados do Sidra com Python.

---

In [18]:
import watermark
%reload_ext watermark
%watermark -a "Lucas Falcão" --iversions

Author: Lucas Falcão

pandas   : 2.2.3
watermark: 2.5.0
sidrapy  : 0.1.4



### 1 - Importação das bibliotecas.

In [19]:
import pandas as pd
import sidrapy

### 2 - Extração a base de dados.

In [20]:
base = sidrapy.get_table(table_code="1086",
                         territorial_level="3",
                         ibge_territorial_code="all",
                         variable="283",
                         period="all")

### 3 - Manipulação da base bruta.

Visualização inicial da BASE.

In [21]:
base.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N,D5C,D5N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Unidade da Federação (Código),Unidade da Federação,Trimestre (Código),Trimestre,Variável (Código),Variável,Tipo de inspeção (Código),Tipo de inspeção,Referência temporal (Código),Referência temporal
1,3,Unidade da Federação,25,Mil litros,54392,11,Rondônia,199701,1º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre
2,3,Unidade da Federação,25,Mil litros,54822,11,Rondônia,199702,2º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre
3,3,Unidade da Federação,25,Mil litros,68027,11,Rondônia,199703,3º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre
4,3,Unidade da Federação,25,Mil litros,84671,11,Rondônia,199704,4º trimestre 1997,283,"Quantidade de leite cru, resfriado ou não, ind...",118225,Total,115236,Total do trimestre


Definição do cabeçalho como colula.

In [22]:
base.columns = base.iloc[0]

Fatiando a base e selecionando as colunas necessárias.

In [23]:
df = base.iloc[1:,[7,6,4]].copy()

Visualização

In [24]:
df.head()

,Trimestre (Código),Unidade da Federação,Valor
1,199701,Rondônia,54392
2,199702,Rondônia,54822
3,199703,Rondônia,68027
4,199704,Rondônia,84671
5,199801,Rondônia,67600


Criação da coluna 'DATA', a partir da coluna 'Trimestre (Código)'

In [25]:
df['DATA'] = pd.to_datetime(df['Trimestre (Código)'].str.slice(0,4) + '-' + df['Trimestre (Código)'].str.slice(4,6))

Dropando a coluna 'Trimestre (Código)', pois a coluna 'DATA' já foi construída.

In [26]:
df = df.drop(columns='Trimestre (Código)')

Renomeação das colunas, e alteração das posições.

In [27]:
df = df.rename(columns={'Unidade da Federação':'UF', 'Valor':'VALOR'})[['DATA','UF','VALOR']]

Transformando a coluna de 'VALOR' em inteiro.

In [28]:
df['VALOR'] = pd.to_numeric(df['VALOR'], errors='coerce').fillna(0).astype(int)

In [29]:
df['UF'] = df['UF'].astype('string')

### 4 - Conferindo os dtypes e se existe valores nulos/ausentes.

Verificando se há valores nulos/ausentes nas colunas.

In [30]:
df.isnull().sum()

0
DATA     0
UF       0
VALOR    0
dtype: int64

Conferindo os tipos de dados.

In [31]:
df.dtypes

0
DATA     datetime64[ns]
UF       string[python]
VALOR             int64
dtype: object

Visualização dos dados filtrados e limpos.

In [32]:
df.head()

,DATA,UF,VALOR
1,1997-01-01,Rondônia,54392
2,1997-02-01,Rondônia,54822
3,1997-03-01,Rondônia,68027
4,1997-04-01,Rondônia,84671
5,1998-01-01,Rondônia,67600


### Análise:
##### Após o processo de extração, manipulação e limpeza, os dados estão prontos para serem utilizados para ralizar análises.

Verificando um ranking entre os Estados do Brasil, referente a produção de leite industrializado.

In [33]:
ranking_estados = df.sort_values('DATA').groupby('UF').tail(1).sort_values('VALOR', ascending=False).reset_index(drop=True)

In [34]:
ranking_estados.head(27)

,DATA,UF,VALOR
0,2024-02-01,Minas Gerais,1422828
1,2024-02-01,Paraná,878614
2,2024-02-01,Santa Catarina,744527
3,2024-02-01,Rio Grande do Sul,649321
4,2024-02-01,São Paulo,548447
5,2024-02-01,Goiás,510394
6,2024-02-01,Bahia,146762
7,2024-02-01,Rondônia,133801
8,2024-02-01,Sergipe,127212
9,2024-02-01,Rio de Janeiro,115777
